In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=123)

import os

### Notes:
- To run this code you need the multi-task-NLP framework XXX

In [2]:
output_path = "../../data-ceph/arguana/argmining22-sharedtask/models/multitask" #replace this with your equivelant path..

#### Prepare the files for training:

- Here we prepare the data into tsv format with the corrsponding columns, so the parser of the frmaework can generate the json files required for training.

In [3]:
taska_training_df = pd.read_csv('../data/TaskA_train.csv')
taska_valid_df    = pd.read_csv('../data/TaskA_dev.csv')
taska_test_df     = pd.read_csv('../data/TaskA_test.csv') #Replace this with the path to the test file

taska_training_df.insert(loc=0,column='row_num',value=np.arange(len(taska_training_df)))
taska_valid_df.insert(loc=0,column='row_num',value=np.arange(len(taska_valid_df)))
taska_test_df.insert(loc=0,column='row_num',value=np.arange(len(taska_test_df)))

#The </s></s>  is the separator used in the pre-trained nli model..
taska_training_df['input_txt'] = taska_training_df.apply(lambda x: '{}:{}  </s></s> {} '.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)
taska_valid_df['input_txt']    = taska_valid_df.apply(lambda x: '{}:{} </s></s> {}'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)
taska_test_df['input_txt']     = taska_test_df.apply(lambda x: '{}:{} </s></s> {}'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)

taska_validity_train_df = taska_training_df[taska_training_df.Validity != 0].copy()
taska_validity_valid_df = taska_valid_df[taska_valid_df.Validity != 0].copy()
taska_validity_test_df  = taska_test_df[taska_test_df.Validity != 0].copy()

taska_validity_train_df['label'] = taska_validity_train_df.Validity.apply(lambda x : "valid" if x == 1 else "invalid")
taska_validity_valid_df['label'] = taska_validity_valid_df.Validity.apply(lambda x : "valid" if x == 1 else "invalid")
taska_validity_test_df['label']  = taska_validity_test_df.Validity.apply(lambda x  : "valid" if x == 1 else "invalid")


taska_novelty_train_df = taska_training_df[taska_training_df.Novelty != 0].copy()
taska_novelty_valid_df = taska_valid_df[taska_valid_df.Novelty != 0].copy()
taska_novelty_test_df  = taska_test_df[taska_test_df.Novelty != 0].copy()

#Balancing the data for novelty task..
taska_novelty_train_balanced_df, y = ros.fit_resample(taska_novelty_train_df, taska_novelty_train_df['Novelty'])
taska_novelty_train_balanced_df['Novelty'] = y

taska_novelty_train_df['label'] = taska_novelty_train_df.Novelty.apply(lambda x : "novel" if x == 1 else "conservative")
taska_novelty_train_balanced_df['label'] = taska_novelty_train_balanced_df.Novelty.apply(lambda x : "novel" if x == 1 else "conservative")
taska_novelty_valid_df['label'] = taska_novelty_valid_df.Novelty.apply(lambda x : "novel" if x == 1 else "conservative")
taska_novelty_test_df['label']  = taska_novelty_test_df.Novelty.apply(lambda x  : "novel" if x == 1 else "conservative")

In [4]:
taska_validity_train_df.label.value_counts()

valid      401
invalid    320
Name: label, dtype: int64

In [5]:
taska_novelty_train_df.label.value_counts()

conservative    595
novel           123
Name: label, dtype: int64

In [6]:
taska_novelty_train_balanced_df.label.value_counts()

conservative    595
novel           595
Name: label, dtype: int64

In [ ]:
taska_validity_train_df[['row_num',  'label', 'input_txt']].to_csv('../data/multitask_data/validity_training_df.tsv', sep='\t', header=False, index=False)
taska_validity_valid_df[['row_num', 'label', 'input_txt' ]].to_csv('../data/multitask_data/validity_valid_df.tsv', sep='\t', header=False, index=False)
taska_validity_test_df[['row_num', 'label', 'input_txt' ]].to_csv('../data/multitask_data/validity_test_df.tsv', sep='\t', header=False, index=False)

taska_novelty_train_df[['row_num', 'label', 'input_txt']].to_csv('../data/multitask_data/novelty_training_df.tsv', sep='\t', header=False, index=False)
taska_novelty_train_balanced_df[['row_num', 'label', 'input_txt']].to_csv('../data/multitask_data/novelty_training_balanced_df.tsv', sep='\t', header=False, index=False)
taska_novelty_valid_df[['row_num', 'label', 'input_txt']].to_csv('../data/multitask_data/novelty_valid_df.tsv', sep='\t', header=False, index=False)
taska_novelty_test_df[['row_num', 'label', 'input_txt']].to_csv('../data/multitask_data/novelty_test_df.tsv', sep='\t', header=False, index=False)

### Experiments with NLI-based Model:

#### Trying different learning rates:

Best lr is 2e-5

In [10]:
for lr in [0.000002, 0.000005, 0.00002, 0.00005]:
    ! sh ./src-py/run_nli_based_multitask_experiment.sh ../data/multitask_data/nli_based_multitask.yml \
                                          ../data/multitask_data/ \
                                          ../../data-ceph/arguana/argmining22-sharedtask/models/multitask/nli_model/multitask/ \
                                          /var/argmining-sharedtask/nli-multitask-$lr\
                                          roberta-large-mnli_prepared_data $lr 10

Preparing data...
task object created from task file...
loading file https://huggingface.co/roberta-large-mnli/resolve/main/vocab.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/64a1d72b2bd05b0aff1a4dd9e7a90a6eea0312b4f914e80b0a923aa8f72219bd.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-large-mnli/resolve/main/merges.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/425529714b758f50b6d3f93f8093d859856fd41cf1cec7c8edf2ab44aee632b6.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/roberta-large-mnli/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/roberta-large-mnli/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/roberta-large-mnli/resolve/main/tokenizer_config.json from cache at None
loading configurati

#### Joint learning of two tasks with Novelty having higher weight:

In [25]:
! sh ./src-py/run_nli_based_multitask_experiment.sh ../data/multitask_data/nli_based_multitask_for_novelty.yml \
                                      ../data/multitask_data/ \
                                      ../../data-ceph/arguana/argmining22-sharedtask/models/multitask/nli_model/multitask-for-novelty/70 \
                                      /var/argmining-sharedtask/nli-multitask-for-novelty/70\
                                      roberta-large-mnli_prepared_data 2e-5 15

Preparing data...
^C
Traceback (most recent call last):
  File "../../multi-task-NLP/data_preparation.py", line 5, in <module>
    from keras.preprocessing.sequence import pad_sequences
  File "/usr/local/lib/python3.8/dist-packages/keras/__init__.py", line 21, in <module>
    from tensorflow.python import tf2
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/__init__.py", line 41, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/__init__.py", line 108, in <module>
    from tensorflow.python.platform import test
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/platform/test.py", line 24, in <module>
    from tensorflow.python.framework import test_util as _test_util
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/test_util.py", line 37, in <module>
    from absl.testing import parameterized
  File "/usr/local/lib/python3.8/dist-packages/absl

#### Joint learning of two tasks with Validity having higher weight:

In [24]:
! sh ./src-py/run_nli_based_multitask_experiment.sh ../data/multitask_data/nli_based_multitask_for_validity.yml \
                                      ../data/multitask_data/ \
                                      ../../data-ceph/arguana/argmining22-sharedtask/models/multitask/nli_model/multitask-for-validity/90 \
                                      /var/argmining-sharedtask/nli-multitask-for-validity/90\
                                      roberta-large-mnli_prepared_data 5e-6 15

Preparing data...
task object created from task file...
loading file https://huggingface.co/roberta-large-mnli/resolve/main/vocab.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/64a1d72b2bd05b0aff1a4dd9e7a90a6eea0312b4f914e80b0a923aa8f72219bd.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-large-mnli/resolve/main/merges.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/425529714b758f50b6d3f93f8093d859856fd41cf1cec7c8edf2ab44aee632b6.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/roberta-large-mnli/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/roberta-large-mnli/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/roberta-large-mnli/resolve/main/tokenizer_config.json from cache at None
loading configurati

#### Single novelty task:

In [ ]:
#no balanced data
! sh ./src-py/run_nli_based_multitask_experiment.sh ../data/multitask_data/nli_based_novelty_task.yml \
                                          ../data/multitask_data/ \
                                          ../../data-ceph/arguana/argmining22-sharedtask/models/multitask/nli_model/novelty/ \
                                          /var/argmining-sharedtask/nli-novelty\
                                          roberta-large-mnli_prepared_data 2e-5 10

In [3]:
#with balanced data
! sh ./src-py/run_nli_based_multitask_experiment.sh ../data/multitask_data/nli_based_novelty_task.yml \
                                          ../data/multitask_data/ \
                                          ../../data-ceph/arguana/argmining22-sharedtask/models/multitask/nli_model/balanced-novelty/ \
                                          /var/argmining-sharedtask/nli-balanced-novelty\
                                          roberta-large-mnli_prepared_data 2e-5 10

Preparing data...
task object created from task file...
loading file https://huggingface.co/roberta-large-mnli/resolve/main/vocab.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/64a1d72b2bd05b0aff1a4dd9e7a90a6eea0312b4f914e80b0a923aa8f72219bd.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-large-mnli/resolve/main/merges.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/425529714b758f50b6d3f93f8093d859856fd41cf1cec7c8edf2ab44aee632b6.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/roberta-large-mnli/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/roberta-large-mnli/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/roberta-large-mnli/resolve/main/tokenizer_config.json from cache at None
loading configurati

#### Single Validity task:

In [18]:
! sh ./src-py/run_nli_based_multitask_experiment.sh ../data/multitask_data/nli_based_validity_task.yml \
                                          ../data/multitask_data/ \
                                          ../../data-ceph/arguana/argmining22-sharedtask/models/multitask/nli_model/validity/ \
                                          /var/argmining-sharedtask/nli-validity\
                                          roberta-large-mnli_prepared_data 5e-6 10

Preparing data...
task object created from task file...
loading file https://huggingface.co/roberta-large-mnli/resolve/main/vocab.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/64a1d72b2bd05b0aff1a4dd9e7a90a6eea0312b4f914e80b0a923aa8f72219bd.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-large-mnli/resolve/main/merges.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/425529714b758f50b6d3f93f8093d859856fd41cf1cec7c8edf2ab44aee632b6.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/roberta-large-mnli/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/roberta-large-mnli/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/roberta-large-mnli/resolve/main/tokenizer_config.json from cache at None
loading configurati

### Perform prediction:

NOTE: There is still an issue when loading the model through the infer file. The results here are not reflective of the model performance. For now look at the logged classification_f1_score during training....

In [48]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
import sys
import glob
import json
sys.path.append('../../multi-task-NLP/')
from sklearn.metrics import f1_score, precision_score, recall_score

In [69]:
%autoreload
from infer_pipeline import inferPipeline

In [70]:
#bert_model = inferPipeline(modelPath = '../data/multitask_data/bert_model/multi_task_model_10_995.pt', maxSeqLen = 512)
nli_model  = inferPipeline(modelPath = '../data/multitask_data/nli_model/multi_task_model_4_455.pt', maxSeqLen = 512, load_pretrained_classifiers=True)

loading file https://huggingface.co/roberta-large-mnli/resolve/main/vocab.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/64a1d72b2bd05b0aff1a4dd9e7a90a6eea0312b4f914e80b0a923aa8f72219bd.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-large-mnli/resolve/main/merges.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/425529714b758f50b6d3f93f8093d859856fd41cf1cec7c8edf2ab44aee632b6.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/roberta-large-mnli/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/roberta-large-mnli/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/roberta-large-mnli/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/roberta-large-mnli/resolv

In [71]:
#bert_preds = bert_model.infer(taska_valid_df['input_txt'].tolist(), ['TaskA', 'TaskB']) 
nli_preds  = nli_model.infer(taska_valid_df['input_txt'].tolist(), ['TaskB']) 

Eval: 100%|██████████| 26/26 [00:01<00:00, 17.63it/s]


In [25]:
#taska_valid_df['bert_pred_validity'] = [x['TaskB'][0] for x in bert_preds]
#taska_valid_df['bert_pred_novelty']  = [x['TaskA'][0] for x in bert_preds]
# taska_valid_df['nli_pred_validity']  = [1 if x['TaskB'][0] == 'ENTAILMENT' else 0 for x in nli_preds]
# taska_valid_df['nli_pred_novelty']   = [1 if x['TaskA'][0] == 'NEUTRAL' else 0 for x in nli_preds]

In [44]:
#print('BERT Validity: ', f1_score(taska_valid_df.Validity.tolist(), taska_valid_df.bert_pred_validity.tolist(), average='macro'))
#print('BERT Novelty: ', f1_score(taska_valid_df.Novelty.tolist(), taska_valid_df.bert_pred_novelty.tolist(), average='macro'))

print('NLI Validity: ', f1_score(taska_valid_df.Validity.tolist(), taska_valid_df.nli_pred_validity.tolist(), average='macro'))
print('NLI Novelty: ', f1_score(taska_valid_df.Novelty.tolist(), taska_valid_df.nli_pred_novelty.tolist(), average='macro'))

NLI Validity:  0.03526121266606733
NLI Novelty:  0.19248826291079815
